# Artificial neural network

In [1]:
#imports
from preprocessing.preprocessing import get_preprocessed_brfss_dataset
from preprocessing.neural_network_preprocessing import get_number_of_numerical_features, NeuralNetworkPreprocessor, CATEGORICAL_COLUMNS, NUMERICAL_COLUMNS
from visualization.general_plots import plot_class_frequencies
from visualization.neural_network_plots import plot_loss, plot_accuracy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
use_mps = True
use_cuda = False

device = torch.device("cpu")

if torch.has_mps and use_mps:
    device = torch.device("mps")
    print("Torch mps activated")

if torch.has_cuda and use_cuda:
    device = torch.device("cuda")
    print("Torch cuda activated")

Torch mps activated


Looking at the dataset it becomes clear that it is imbalanced

In [ ]:
train_dataset, target = get_preprocessed_brfss_dataset()
plot_class_frequencies(target)

In [ ]:
preprocessor = NeuralNetworkPreprocessor()
data_train, data_test, target_train, target_test = preprocessor.get_preprocessed_dataset_for_neural_network()
target_train.head(10)

In [ ]:
#constants
batch_size = 128
output_size = target_train.shape[1]
embedding_sizes = preprocessor.get_embedding_sizes()
embedding_input_size = sum((nf for ni, nf in embedding_sizes))
numerical_input_size = get_number_of_numerical_features()
input_size = embedding_input_size + numerical_input_size
print(f"Input size: {input_size}")
print(f"Output size: {output_size}")
print(f"Embedding sizes: {embedding_sizes}")

In [ ]:
for class_label in range(4):
    predicted = [0,0,0,0]
    predicted[class_label] = 1
    correct = 0
    total = 0
    for i in range(target_train.shape[0]):
        labels = target_train.values[i]
        labels = [int(element) for element in labels]
        total += 1
        if predicted == labels:
            correct +=1
    print(f'Accuracy class {predicted} : {100 * correct / total} %')


In [ ]:
class CustomBrfssDataset(data_utils.Dataset):
    def __init__(self, dataset_complete, dataset_target):
        self.data_categorical = dataset_complete[CATEGORICAL_COLUMNS]
        self.data_categorical = torch.tensor(np.array(self.data_categorical)).int()

        self.data_numerical = dataset_complete[NUMERICAL_COLUMNS]
        self.data_numerical = torch.tensor(np.array(self.data_numerical)).float()

        self.target = torch.tensor(np.array(dataset_target)).float()

    def __len__(self):
            return len(self.data_categorical)

    def __getitem__(self, idx):
            return self.data_categorical[idx], self.data_numerical[idx], self.target[idx]

Create torch data loader

In [ ]:
train_dataset = CustomBrfssDataset(data_train, target_train)
test_dataset = CustomBrfssDataset(data_test, target_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Create artificial net, define loss function and define optimizer

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
        self.batch_norm_numerical = nn.BatchNorm1d(numerical_input_size)
        self.dropout_embedding = nn.Dropout(0.3)

        self.fc1 = nn.Linear(input_size, 500)
        self.bn1 = nn.BatchNorm1d(500)
        self.do1 = nn.Dropout(0.8)
        self.fc2 = nn.Linear(500, 1000)
        self.bn2 = nn.BatchNorm1d(1000)
        self.do2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(1000, output_size)

    def forward(self, input_categorical, input_numerical):
        embedding_layers = []
        for index,e in enumerate(self.embeddings):
            embedding_layers.append(e(input_categorical[:,index]))
        x_categorical = torch.cat(embedding_layers, 1)
        x_categorical = self.dropout_embedding(x_categorical)

        x_numerical = self.batch_norm_numerical(input_numerical)
        x = torch.cat([x_categorical, x_numerical], 1)

        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu_(x)
        x = self.do1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu_(x)
        x = self.do2(x)
        x = self.fc3(x)
        x = F.softmax(x)
        return x


net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.002)

In [ ]:
def get_accuracy(data_loader):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
            inputs_categorical, inputs_numerical, labels = data
            inputs_categorical = inputs_categorical.to(device)
            inputs_numerical = inputs_numerical.to(device)
            labels = labels.to(device)

            outputs = net(inputs_categorical, inputs_numerical)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct/total

def get_loss(data_loader):
    running_loss = 0.0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            total += labels.size(0)

    return running_loss/total

Training loop

In [ ]:
def train_network():
    loss_values = []
    accuracy_values = []

    for epoch in range(5):

        running_loss = 0.0

        net.train()
        for i, data in enumerate(train_loader, 0):

            inputs_categorical, inputs_numerical, labels = data
            inputs_categorical = inputs_categorical.to(device)
            inputs_numerical = inputs_numerical.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs_categorical, inputs_numerical)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:  # print every 2000 batches
                print(f'[{epoch + 1}, {(i + 1) * batch_size}] loss: {running_loss / i:.3f}')

            pass

        loss_values.append(running_loss/len(train_loader))
        train_accuracy = 100 * get_accuracy(train_loader)
        accuracy_values.append(train_accuracy)

        print(f"Epoch {epoch} loss: {str(running_loss/len(train_loader))}")
        print(f'Accuracy epoch {epoch}: {train_accuracy} %')

    print('Finished Training')
    return loss_values, accuracy_values

### Train model without under or oversampling

In [ ]:
loss_values, accuracy_values = train_network()

In [ ]:
print(f'Accuracy : {100 * get_accuracy(test_loader)} %')

In [ ]:
plot_loss(loss_values)

In [ ]:
plot_accuracy(accuracy_values)

### Train model with undersampling

In [ ]:
net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.002)

data_train, data_test, target_train, target_test = preprocessor.get_preprocessed_dataset_for_neural_network_undersampled()

train_dataset = CustomBrfssDataset(data_train, target_train)
test_dataset = CustomBrfssDataset(data_test, target_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

loss_values, accuracy_values = train_network()